In [3]:
!pip install wbgapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mount Drive dan Import Library

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import wbgapi as wb

In [5]:
# Youth are defined as persons ages 15 to 24; young adults are those ages 25 to 29; and adults are those ages 25 and above.

# SL.UEM.NEET.ZS	Share of youth not in education, employment or training, total (% of youth population)

# SP.POP.1564.TO	Population ages 15-64, total

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Get Dataset
Membangun dataframe dari dataset yang diambil dari World Bank dan file CSV yang diunduh dari World Population Review.
Data yang diambil mulai dari tahun 2017-2021 untuk World Bank.
Dataset dibangun berdasarkan indikator.

In [7]:
# List Nama Negara ASEAN
asean = ['Brunei', 'Myanmar', 'Cambodia', 'Timor-Leste', 'Indonesia', 'Laos', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam']
asean

['Brunei',
 'Myanmar',
 'Cambodia',
 'Timor-Leste',
 'Indonesia',
 'Laos',
 'Malaysia',
 'Philippines',
 'Singapore',
 'Thailand',
 'Vietnam']

In [8]:
# Ambil Dataset Average IQ by Country dan ambil data negara ASEAN
df_iq = pd.read_csv('/content/drive/MyDrive/TETRIS Batch 3/capstone project/average-iq-by-country-2023.csv')
df_iq = df_iq[['rank','country','iq']].loc[df_iq['country'].isin(asean)]
df_iq.rename(columns = {'country':'Nama Negara','rank':'Rank IQ', 'iq':'IQ'}, inplace = True)
df_iq

,Rank IQ,Nama Negara,IQ
2,3,Singapore,105.89
14,15,Cambodia,99.75
51,52,Myanmar,91.18
59,60,Vietnam,89.53
63,64,Thailand,88.87
72,73,Malaysia,87.58
73,74,Brunei,87.58
110,111,Philippines,81.64
113,114,Laos,80.99
129,130,Indonesia,78.49


In [9]:
# Mencari kode Negara ASEAN yang digunakan World Bank
nama_kode_negara = pd.DataFrame(wb.economy.coder(asean).items(),columns=['Nama Negara','code'])
nama_kode_negara.head()

,Nama Negara,code
0,Brunei,BRN
1,Myanmar,MMR
2,Cambodia,KHM
3,Timor-Leste,TLS
4,Indonesia,IDN


In [10]:
# Mencari Income level tiap negara berdasarkan data di World Bank

# HIC	High income			
# LIC	Low income		
# LMC	Lower middle income		
# LMY	Low & middle income		
# MIC	Middle income		
# UMC	Upper middle income

income_level = wb.economy.DataFrame(nama_kode_negara['code'].tolist()).reset_index()
income_level = income_level.drop(columns=['name','aggregate',	'longitude', 'latitude',	'region',	'adminregion',	'lendingType', 'capitalCity'])
income_level.rename(columns = {'id':'code', 'incomeLevel':'Income Level'}, inplace = True)
income_level.head()

,code,Income Level
0,BRN,HIC
1,KHM,LMC
2,IDN,LMC
3,LAO,LMC
4,MYS,UMC


In [11]:
# Menggabungkan dataframe nama_kode_negara, income_level, df_iq
nama_kode_negara = pd.merge(nama_kode_negara, income_level, on="code")
info = pd.merge(nama_kode_negara, df_iq, on="Nama Negara")
info.head()

,Nama Negara,code,Income Level,Rank IQ,IQ
0,Brunei,BRN,HIC,74,87.58
1,Myanmar,MMR,LMC,52,91.18
2,Cambodia,KHM,LMC,15,99.75
3,Timor-Leste,TLS,LMC,132,78.49
4,Indonesia,IDN,LMC,130,78.49


In [12]:
info['code'].tolist()

['BRN', 'MMR', 'KHM', 'TLS', 'IDN', 'LAO', 'MYS', 'PHL', 'SGP', 'THA', 'VNM']

In [13]:
df_youth = pd.read_csv('/content/drive/MyDrive/TETRIS Batch 3/capstone project/youth.csv')
# df_youth = df_youth[df_youth['Country Name'].isin(asean)]
df_youth.head()

,code,Year,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,BRN,2017,16815,18112,16545,17857,69329,19.969999,13846
1,MMR,2017,2251712,2350164,2197402,2305666,9104944,16.590000,1510511
2,KHM,2017,701666,706196,659080,689642,2756584,6.460000,178076
3,TLS,2017,71122,75007,67084,60935,274148,31.170000,91852
4,IDN,2017,10722773,11306337,10675063,11043839,43748012,21.450001,9383949


In [14]:
negara_info = pd.merge(info, df_youth, on="code")
negara_info.head()

,Nama Negara,code,Income Level,Rank IQ,IQ,Year,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,Brunei,BRN,HIC,74,87.58,2017,16815,18112,16545,17857,69329,19.969999,13846
1,Brunei,BRN,HIC,74,87.58,2018,16852,18002,16669,17772,69295,20.719999,14358
2,Brunei,BRN,HIC,74,87.58,2019,16777,17815,16575,17742,68909,18.680000,12873
3,Brunei,BRN,HIC,74,87.58,2020,16574,17546,16557,17786,68463,21.430000,14672
4,Brunei,BRN,HIC,74,87.58,2021,16361,17260,16545,17850,68016,18.680000,12706


In [15]:
# Menyiapkan indikator yang digunakan untuk mencari data di World Bank
dict_indikator={
    "SP.POP.TOTL": "Total Populasi", # SP.POP.TOTL - Total Population
    "NY.GDP.PCAP.CD" : "Pertumbuhan pendapatan negara", # NY.GDP.PCAP.CD - income growth for countries
    "NY.GDP.MKTP.CD" : "PDB dalam USD", # NY.GDP.MKTP.CD - GDP in USD
    "SE.XPD.TOTL.GD.ZS"	: "Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)", # SE.XPD.TOTL.GD.ZS	- Government expenditure on education, total (% of GDP)
    "SE.COM.DURS" : "Durasi Wajib Belajar", # SE.COM.DURS - Compulsory Education Duration
}

In [16]:
# Fungsi untuk mengambil dataset berdasarkan kode dan tanggal
def getDataset(kode,thn_awal,thn_akhir):
  # mengambil data yang diperlukan dari World bank
  df = wb.data.DataFrame(list(dict_indikator.keys()), kode, time=range(thn_awal,thn_akhir)).reset_index()

  # Membuat list nama kolom
  kolom = []
  for k in df['series'].tolist():
    for key, value in dict_indikator.items():
      if k == key:
        kolom.append(value)

  # Membuat dataframe yang baru dengan nama kolom yg baru
  df_new = pd.DataFrame(columns=kolom)

  # Drop 'series' pada dataframe lama
  df.drop('series',axis=1,inplace=True)

  # Menambahkan data ke dataset baru
  for i in df.columns.values.tolist():
    df_new.loc[i[2:]]= df[i].tolist()

  # Menambah Kolom yang diperlukan
  df_new = df_new.reset_index()
  df_new = df_new.rename(columns ={'index':'Year'})

  nama = negara_info.loc[negara_info['code'] == kode]
  df_new['Nama Negara'] = nama['Nama Negara'].values[0]

  return df_new

In [17]:
# Mengambil Data Negara Brunei 
data_Brunei = getDataset('BRN',2017,2022)
data_Brunei

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,1.212810e+10,28186.803027,9.0,NaN,430276.0,Brunei
1,2018,1.356735e+10,31241.453955,9.0,NaN,434274.0,Brunei
2,2019,1.346942e+10,30748.737487,9.0,NaN,438048.0,Brunei
3,2020,1.200583e+10,27179.411985,9.0,NaN,441725.0,Brunei
4,2021,1.400657e+10,31449.076562,9.0,NaN,445373.0,Brunei


In [18]:
# Mengambil Data Negara Myanmar 
data_Myanmar = getDataset('MMR',2017,2022)
data_Myanmar

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,6.144939e+10,1175.202557,5.0,2.32634,52288341.0,Myanmar
1,2018,6.714473e+10,1274.915657,5.0,2.15748,52666014.0,Myanmar
2,2019,6.869776e+10,1295.201448,5.0,2.13987,53040212.0,Myanmar
3,2020,7.893026e+10,1477.452870,5.0,NaN,53423198.0,Myanmar
4,2021,6.509175e+10,1209.926942,5.0,NaN,53798084.0,Myanmar


In [19]:
# Mengambil Data Negara Cambodia 
data_Cambodia = getDataset('KHM',2017,2022)
data_Cambodia

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,2.217720e+10,1400.899260,NaN,2.316152,15830689.0,Cambodia
1,2018,2.457175e+10,1533.315985,NaN,2.578059,16025238.0,Cambodia
2,2019,2.708939e+10,1671.385385,NaN,2.896630,16207746.0,Cambodia
3,2020,2.587280e+10,1577.911747,NaN,3.126307,16396860.0,Cambodia
4,2021,2.696106e+10,1625.235020,NaN,NaN,16589023.0,Cambodia


In [20]:
# Mengambil Data Negara Timor-Leste
data_TimorLeste = getDataset('TLS',2017,2022)
data_TimorLeste

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,1.595724e+09,1283.525850,9.0,5.144509,1243235.0,Timor-Leste
1,2018,1.563888e+09,1239.366122,9.0,4.819879,1261845.0,Timor-Leste
2,2019,2.028552e+09,1584.263844,9.0,4.137038,1280438.0,Timor-Leste
3,2020,2.158393e+09,1660.308340,9.0,4.153062,1299995.0,Timor-Leste
4,2021,3.621222e+09,2741.393931,9.0,NaN,1320942.0,Timor-Leste


In [21]:
# Mengambil Data Negara Indonesia
data_Indonesia = getDataset('IDN',2017,2022)
data_Indonesia

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,1.015619e+12,3839.785069,9.0,2.66998,264498852.0,Indonesia
1,2018,1.042272e+12,3902.661668,9.0,3.00000,267066843.0,Indonesia
2,2019,1.119100e+12,4151.227543,9.0,2.84185,269582878.0,Indonesia
3,2020,1.058689e+12,3894.272202,9.0,3.49000,271857970.0,Indonesia
4,2021,1.186093e+12,4332.709281,9.0,NaN,273753191.0,Indonesia


In [22]:
# Mengambil Data Negara Laos
data_Laos = getDataset('LAO',2017,2022)
data_Laos

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,1.707116e+10,2439.463355,9.0,2.497973,6997917.0,Laos
1,2018,1.814165e+10,2553.361866,9.0,2.335012,7105006.0,Laos
2,2019,1.874056e+10,2598.505523,9.0,2.475330,7212053.0,Laos
3,2020,1.898180e+10,2593.355097,9.0,2.299760,7319399.0,Laos
4,2021,1.882715e+10,2535.623432,9.0,NaN,7425057.0,Laos


In [23]:
# Mengambil Data Negara Malaysia
data_Malaysia = getDataset('MYS',2017,2022)
data_Malaysia

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,3.191122e+11,9979.800841,6.0,4.67531,31975806.0,Malaysia
1,2018,3.587916e+11,11074.064095,6.0,4.47865,32399271.0,Malaysia
2,2019,3.651751e+11,11132.023934,6.0,4.15661,32804020.0,Malaysia
3,2020,3.373379e+11,10160.783247,6.0,3.91697,33199993.0,Malaysia
4,2021,3.729810e+11,11109.261839,6.0,NaN,33573874.0,Malaysia


In [24]:
# Mengambil Data Negara Philippines
data_Philippines = getDataset('PHL',2017,2022)
data_Philippines

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,3.284809e+11,3077.435640,11.0,3.971154,106738501.0,Philippines
1,2018,3.468421e+11,3194.674521,13.0,3.864053,108568836.0,Philippines
2,2019,3.768233e+11,3413.847924,13.0,3.355792,110380804.0,Philippines
3,2020,3.617511e+11,3224.422551,13.0,3.734811,112190977.0,Philippines
4,2021,3.940864e+11,3460.530963,13.0,NaN,113880328.0,Philippines


In [25]:
# Mengambil Data Negara Singapore
data_Singapore = getDataset('SGP',2017,2022)
data_Singapore

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,3.431934e+11,61150.727197,6.0,2.768260,5612253.0,Singapore
1,2018,3.769981e+11,66859.338345,6.0,2.855980,5638676.0,Singapore
2,2019,3.754727e+11,65831.189431,6.0,2.736210,5703569.0,Singapore
3,2020,3.452959e+11,60729.450349,6.0,2.506660,5685807.0,Singapore
4,2021,3.969869e+11,72794.003023,6.0,2.806462,5453566.0,Singapore


In [26]:
# Mengambil Data Negara Thailand
data_Thailand = getDataset('THA',2017,2022)
data_Thailand

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,4.563570e+11,6436.791746,9.0,3.465322,70898202.0,Thailand
1,2018,5.067546e+11,7124.564544,9.0,3.198600,71127802.0,Thailand
2,2019,5.440811e+11,7630.039610,9.0,3.020620,71307763.0,Thailand
3,2020,4.996818e+11,6990.935503,9.0,3.145711,71475664.0,Thailand
4,2021,5.059470e+11,7066.190546,9.0,NaN,71601103.0,Thailand


In [27]:
# Mengambil Data Negara Vietnam
data_Vietnam = getDataset('VNM',2017,2022)
data_Vietnam

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,2.813536e+11,2992.071742,10.0,4.085540,94033048.0,Vietnam
1,2018,3.101065e+11,3267.225009,10.0,4.167450,94914330.0,Vietnam
2,2019,3.343653e+11,3491.091274,10.0,4.061970,95776716.0,Vietnam
3,2020,3.466158e+11,3586.347302,10.0,4.111617,96648685.0,Vietnam
4,2021,3.661376e+11,3756.489121,10.0,NaN,97468029.0,Vietnam


# Data Preprocessing
Seperti yang bisa kita lihat dari datase yang kita buat mengandung beberapa missing value di beberapa fitur. Ini berarti bahwa data memerlukan beberapa pemrosesan sebelum digunakan untuk analisis.

Sebelum itu, kita akan menyalin dataset agar kita bisa memiliki data sebelum Preprocessimg dan sesudah preprocessing.

## Handling Missing Value

In [28]:
copy_data_Brunei = data_Brunei.copy()
copy_data_Myanmar = data_Myanmar.copy()
copy_data_Cambodia = data_Cambodia.copy()
copy_data_TimorLeste = data_TimorLeste.copy()
copy_data_Indonesia = data_Indonesia.copy()
copy_data_Laos = data_Laos.copy()
copy_data_Malaysia = data_Malaysia.copy()
copy_data_Philippines = data_Philippines.copy()
copy_data_Singapore = data_Singapore.copy()
copy_data_Thailand = data_Thailand.copy()
copy_data_Vietnam = data_Vietnam.copy()

In [29]:
copy_data_Brunei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  0 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [30]:
copy_data_Myanmar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  3 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [31]:
copy_data_Cambodia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         0 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [32]:
copy_data_TimorLeste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [33]:
copy_data_Indonesia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [34]:
copy_data_Laos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [35]:
copy_data_Malaysia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [36]:
copy_data_Philippines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [37]:
copy_data_Singapore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  5 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [38]:
copy_data_Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [39]:
copy_data_Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  4 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


Missing Value pada data Brunei ditemukan pada salah satu kolom. Untuk mengatasi hal ini, saya memutuskan untuk melihat data pada 5 tahun sebelum 2017 dengan mengambil data lagi di World Bank. Dari data tersebut diambil nilai meannya.

In [40]:
mean_brunei = wb.data.DataFrame('SE.XPD.TOTL.GD.ZS', 'BRN', time=range(2013,2018)).reset_index()
mean_brunei.drop('economy',axis=1, inplace=True)
mean_brunei = mean_brunei.transpose()[0].mean()
mean_brunei

3.889299869537355

In [41]:
copy_data_Brunei['Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)'].fillna((mean_brunei), inplace=True)

Untuk data Cambodia memiliki missing value pada kolom Durasi Wajib Belajar. Bila kita melihat tahun sebelum-sebelumnya jg masih berupa Missing Value. Maka dari itu, saya mencari sendiri durasi belajar yang ditetapkan di Kamboja. Pada jurnal yang ditemukan, diketahui bahwa Kamboja menetapkan durasi wajib belajar selama 12 tahun.

In [42]:
copy_data_Cambodia['Durasi Wajib Belajar'].fillna(12, inplace=True)
copy_data_Cambodia

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara
0,2017,2.217720e+10,1400.899260,12.0,2.316152,15830689.0,Cambodia
1,2018,2.457175e+10,1533.315985,12.0,2.578059,16025238.0,Cambodia
2,2019,2.708939e+10,1671.385385,12.0,2.896630,16207746.0,Cambodia
3,2020,2.587280e+10,1577.911747,12.0,3.126307,16396860.0,Cambodia
4,2021,2.696106e+10,1625.235020,12.0,NaN,16589023.0,Cambodia


Untuk data yang lainnya, missing value akan diisi dengan nilai mean

In [43]:
# Fungsi untuk mengisi missing value dengan mean
def fill_missing_value(df):
  col_list = list(df.columns)
  for col in col_list:
    # Jika kolom tipe datanya Object, maka abaikan
    if col in df.select_dtypes(include=object).columns.values.tolist():
      continue
      
    df[col].fillna((df[col].mean()), inplace=True)
  return df

In [44]:
list_filna = [copy_data_Myanmar, copy_data_Cambodia, copy_data_TimorLeste, copy_data_Indonesia, 
              copy_data_Laos, copy_data_Malaysia, copy_data_Philippines, copy_data_Thailand, copy_data_Vietnam]

In [45]:
for df in list_filna:
  fill_missing_value(df)

In [46]:
copy_data_Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         5 non-null      object 
 1   PDB dalam USD                                                5 non-null      float64
 2   Pertumbuhan pendapatan negara                                5 non-null      float64
 3   Durasi Wajib Belajar                                         5 non-null      float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)  5 non-null      float64
 5   Total Populasi                                               5 non-null      float64
 6   Nama Negara                                                  5 non-null      object 
dtypes: float64(5), object(2)
memory usage: 408.0+ bytes


In [47]:
negara_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column                                                                                  Non-Null Count  Dtype  
---  ------                                                                                  --------------  -----  
 0   Nama Negara                                                                             55 non-null     object 
 1   code                                                                                    55 non-null     object 
 2   Income Level                                                                            55 non-null     object 
 3   Rank IQ                                                                                 55 non-null     int64  
 4   IQ                                                                                      55 non-null     float64
 5   Year                                                                     

## Menambah kolom baru
Kita memiliki kolom 'Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)', dimana data merupakan persen dari data PBD. Maka perlu diperlukan perhitungan untuk mendapat nilai 'Pengeluaran pemerintah untuk pendidikan' yang sebenarnya.

In [48]:
def total_pengeluaran_pendidikan(df):
  df['Total pengeluaran pemerintah untuk pendidikan'] = df['PDB dalam USD']*(df['Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)']/100)
  df['Total pengeluaran pemerintah SELAIN pendidikan'] = df['PDB dalam USD']-df['Total pengeluaran pemerintah untuk pendidikan']

  return df

In [49]:
list_merge = [copy_data_Brunei, copy_data_Myanmar, copy_data_Cambodia, copy_data_TimorLeste, 
          copy_data_Indonesia, copy_data_Laos, copy_data_Malaysia, copy_data_Philippines, 
          copy_data_Singapore, copy_data_Thailand, copy_data_Vietnam]

In [50]:
for i in list_merge:
  total_pengeluaran_pendidikan(i)

In [51]:
copy_data_Brunei

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan
0,2017,1.212810e+10,28186.803027,9.0,3.8893,430276.0,Brunei,4.716984e+08,1.165641e+10
1,2018,1.356735e+10,31241.453955,9.0,3.8893,434274.0,Brunei,5.276750e+08,1.303968e+10
2,2019,1.346942e+10,30748.737487,9.0,3.8893,438048.0,Brunei,5.238662e+08,1.294556e+10
3,2020,1.200583e+10,27179.411985,9.0,3.8893,441725.0,Brunei,4.669426e+08,1.153888e+10
4,2021,1.400657e+10,31449.076562,9.0,3.8893,445373.0,Brunei,5.447575e+08,1.346181e+10


## Merge Dataset

In [52]:
negara_info["Year"] = negara_info["Year"].astype("str")
negara_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column                                                                                  Non-Null Count  Dtype  
---  ------                                                                                  --------------  -----  
 0   Nama Negara                                                                             55 non-null     object 
 1   code                                                                                    55 non-null     object 
 2   Income Level                                                                            55 non-null     object 
 3   Rank IQ                                                                                 55 non-null     int64  
 4   IQ                                                                                      55 non-null     float64
 5   Year                                                                     

In [53]:
Brunei_info = negara_info[negara_info['Nama Negara'] == 'Brunei']
Brunei_info.drop('Nama Negara', axis=1, inplace=True)
Brunei_merge = pd.merge(copy_data_Brunei, Brunei_info, on="Year")
Brunei_merge

<ipython-input-53-51ff3d9bbf53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brunei_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,1.212810e+10,28186.803027,9.0,3.8893,430276.0,Brunei,4.716984e+08,1.165641e+10,BRN,HIC,74,87.58,16815,18112,16545,17857,69329,19.969999,13846
1,2018,1.356735e+10,31241.453955,9.0,3.8893,434274.0,Brunei,5.276750e+08,1.303968e+10,BRN,HIC,74,87.58,16852,18002,16669,17772,69295,20.719999,14358
2,2019,1.346942e+10,30748.737487,9.0,3.8893,438048.0,Brunei,5.238662e+08,1.294556e+10,BRN,HIC,74,87.58,16777,17815,16575,17742,68909,18.680000,12873
3,2020,1.200583e+10,27179.411985,9.0,3.8893,441725.0,Brunei,4.669426e+08,1.153888e+10,BRN,HIC,74,87.58,16574,17546,16557,17786,68463,21.430000,14672
4,2021,1.400657e+10,31449.076562,9.0,3.8893,445373.0,Brunei,5.447575e+08,1.346181e+10,BRN,HIC,74,87.58,16361,17260,16545,17850,68016,18.680000,12706


In [54]:
Myanmar_info = negara_info[negara_info['Nama Negara'] == 'Myanmar']
Myanmar_info.drop('Nama Negara', axis=1, inplace=True)
Myanmar_merge = pd.merge(copy_data_Myanmar, Myanmar_info, on="Year")
Myanmar_merge

<ipython-input-54-734f28cb60fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Myanmar_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,6.144939e+10,1175.202557,5.0,2.326340,52288341.0,Myanmar,1.429522e+09,6.001987e+10,MMR,LMC,52,91.18,2251712,2350164,2197402,2305666,9104944,16.590000,1510511
1,2018,6.714473e+10,1274.915657,5.0,2.157480,52666014.0,Myanmar,1.448634e+09,6.569609e+10,MMR,LMC,52,91.18,2258212,2358455,2217477,2295470,9129614,12.790000,1167678
2,2019,6.869776e+10,1295.201448,5.0,2.139870,53040212.0,Myanmar,1.470043e+09,6.722772e+10,MMR,LMC,52,91.18,2260617,2362041,2204992,2284447,9112097,16.978809,1547126
3,2020,7.893026e+10,1477.452870,5.0,2.207897,53423198.0,Myanmar,1.742698e+09,7.718756e+10,MMR,LMC,52,91.18,2258470,2360501,2196937,2278275,9094183,15.020000,1365947
4,2021,6.509175e+10,1209.926942,5.0,2.207897,53798084.0,Myanmar,1.437159e+09,6.365459e+10,MMR,LMC,52,91.18,2249244,2351085,2197402,2280229,9077960,16.978809,1541330


In [55]:
Cambodia_info = negara_info[negara_info['Nama Negara'] == 'Cambodia']
Cambodia_info.drop('Nama Negara', axis=1, inplace=True)
Cambodia_merge = pd.merge(copy_data_Cambodia, Cambodia_info, on="Year")
Cambodia_merge

<ipython-input-55-32bf6b707694>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cambodia_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,2.217720e+10,1400.899260,12.0,2.316152,15830689.0,Cambodia,5.136576e+08,2.166354e+10,KHM,LMC,15,99.75,701666,706196,659080,689642,2756584,6.46,178076
1,2018,2.457175e+10,1533.315985,12.0,2.578059,16025238.0,Cambodia,6.334744e+08,2.393828e+10,KHM,LMC,15,99.75,707983,714473,690218,661693,2774367,6.46,178076
2,2019,2.708939e+10,1671.385385,12.0,2.896630,16207746.0,Cambodia,7.846793e+08,2.630471e+10,KHM,LMC,15,99.75,708255,715641,671736,642306,2737938,6.46,178076
3,2020,2.587280e+10,1577.911747,12.0,3.126307,16396860.0,Cambodia,8.088630e+08,2.506394e+10,KHM,LMC,15,99.75,713041,723077,659791,631524,2727433,6.46,178076
4,2021,2.696106e+10,1625.235020,12.0,2.729287,16589023.0,Cambodia,7.358447e+08,2.622522e+10,KHM,LMC,15,99.75,722553,738241,659080,635501,2755375,6.46,178076


In [56]:
TimorLeste_info = negara_info[negara_info['Nama Negara'] == 'Timor-Leste']
TimorLeste_info.drop('Nama Negara', axis=1, inplace=True)
TimorLeste_merge = pd.merge(copy_data_TimorLeste, TimorLeste_info, on="Year")
TimorLeste_merge

<ipython-input-56-27ac0e50e8eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TimorLeste_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,1.595724e+09,1283.525850,9.0,5.144509,1243235.0,Timor-Leste,8.209218e+07,1.513632e+09,TLS,LMC,132,78.49,71122,75007,67084,60935,274148,31.17,91852
1,2018,1.563888e+09,1239.366122,9.0,4.819879,1261845.0,Timor-Leste,7.537751e+07,1.488510e+09,TLS,LMC,132,78.49,72587,76880,61190,63210,273867,31.17,91852
2,2019,2.028552e+09,1584.263844,9.0,4.137038,1280438.0,Timor-Leste,8.392196e+07,1.944630e+09,TLS,LMC,132,78.49,73846,78577,63001,65585,281009,31.17,91852
3,2020,2.158393e+09,1660.308340,9.0,4.153062,1299995.0,Timor-Leste,8.963938e+07,2.068753e+09,TLS,LMC,132,78.49,74924,80111,64969,68024,288028,31.17,91852
4,2021,3.621222e+09,2741.393931,9.0,4.563622,1320942.0,Timor-Leste,1.652589e+08,3.455963e+09,TLS,LMC,132,78.49,75756,81298,67084,70541,294679,31.17,91852


In [57]:
Indonesia_info = negara_info[negara_info['Nama Negara'] == 'Indonesia']
Indonesia_info.drop('Nama Negara', axis=1, inplace=True)
Indonesia_merge = pd.merge(copy_data_Indonesia, Indonesia_info, on="Year")
Indonesia_merge

<ipython-input-57-3a7e8cd8c194>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Indonesia_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,1.015619e+12,3839.785069,9.0,2.669980,264498852.0,Indonesia,2.711682e+10,9.885019e+11,IDN,LMC,130,78.49,10722773,11306337,10675063,11043839,43748012,21.450001,9383949
1,2018,1.042272e+12,3902.661668,9.0,3.000000,267066843.0,Indonesia,3.126815e+10,1.011003e+12,IDN,LMC,130,78.49,10696599,11281052,10587048,11137955,43702654,21.740000,9500957
2,2019,1.119100e+12,4151.227543,9.0,2.841850,269582878.0,Indonesia,3.180314e+10,1.087297e+12,IDN,LMC,130,78.49,10700127,11285209,10644508,11195088,43824932,22.450001,9838698
3,2020,1.058689e+12,3894.272202,9.0,3.490000,271857970.0,Indonesia,3.694824e+10,1.021741e+12,IDN,LMC,130,78.49,10740346,11326932,10670575,11221796,43959649,21.799999,9583204
4,2021,1.186093e+12,4332.709281,9.0,3.000458,273753191.0,Indonesia,3.558822e+10,1.150505e+12,IDN,LMC,130,78.49,10815587,11404334,10675063,11227225,44122209,22.450001,9905437


In [58]:
Laos_info = negara_info[negara_info['Nama Negara'] == 'Laos']
Laos_info.drop('Nama Negara', axis=1, inplace=True)
Laos_merge = pd.merge(copy_data_Laos, Laos_info, on="Year")
Laos_merge

<ipython-input-58-f4ce713ab7d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Laos_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,1.707116e+10,2439.463355,9.0,2.497973,6997917.0,Laos,4.264330e+08,1.664473e+10,LAO,LMC,114,80.99,359883,372564,352878,349712,1435037,25.799999,370240
1,2018,1.814165e+10,2553.361866,9.0,2.335012,7105006.0,Laos,4.236098e+08,1.771804e+10,LAO,LMC,114,80.99,359018,371647,344606,353013,1428284,25.799999,370240
2,2019,1.874056e+10,2598.505523,9.0,2.475330,7212053.0,Laos,4.638906e+08,1.827667e+10,LAO,LMC,114,80.99,357142,369361,347500,357345,1431348,25.799999,370240
3,2020,1.898180e+10,2593.355097,9.0,2.299760,7319399.0,Laos,4.365359e+08,1.854526e+10,LAO,LMC,114,80.99,355399,367466,350578,361538,1434981,25.799999,370240
4,2021,1.882715e+10,2535.623432,9.0,2.402019,7425057.0,Laos,4.522316e+08,1.837492e+10,LAO,LMC,114,80.99,355126,367590,352878,364464,1440058,25.799999,370240


In [59]:
Malaysia_info = negara_info[negara_info['Nama Negara'] == 'Malaysia']
Malaysia_info.drop('Nama Negara', axis=1, inplace=True)
Malaysia_merge = pd.merge(copy_data_Malaysia, Malaysia_info, on="Year")
Malaysia_merge

<ipython-input-59-efe6823eb4e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Malaysia_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,3.191122e+11,9979.800841,6.0,4.675310,31975806.0,Malaysia,1.491948e+10,3.041927e+11,MYS,UMC,73,87.58,1349203,1433556,1378715,1497811,5659285,11.800000,667796
1,2018,3.587916e+11,11074.064095,6.0,4.478650,32399271.0,Malaysia,1.606902e+10,3.427226e+11,MYS,UMC,73,87.58,1341861,1427244,1411207,1485707,5666019,12.470000,706553
2,2019,3.651751e+11,11132.023934,6.0,4.156610,32804020.0,Malaysia,1.517891e+10,3.499962e+11,MYS,UMC,73,87.58,1331024,1414952,1398053,1472475,5616504,17.011107,955430
3,2020,3.373379e+11,10160.783247,6.0,3.916970,33199993.0,Malaysia,1.321343e+10,3.241245e+11,MYS,UMC,73,87.58,1320856,1401068,1386927,1461917,5570768,13.630000,759296
4,2021,3.729810e+11,11109.261839,6.0,4.306885,33573874.0,Malaysia,1.606386e+10,3.569171e+11,MYS,UMC,73,87.58,1311531,1387007,1378715,1456031,5533284,17.011107,941273


In [60]:
Philippines_info = negara_info[negara_info['Nama Negara'] == 'Philippines']
Philippines_info.drop('Nama Negara', axis=1, inplace=True)
Philippines_merge = pd.merge(copy_data_Philippines, Philippines_info, on="Year")
Philippines_merge

<ipython-input-60-a69856bad99d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Philippines_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,3.284809e+11,3077.435640,11.0,3.971154,106738501.0,Philippines,1.304448e+10,3.154364e+11,PHL,LMC,111,81.64,5072893,5368559,5017104,4987005,20445561,21.700001,4436687
1,2018,3.468421e+11,3194.674521,13.0,3.864053,108568836.0,Philippines,1.340216e+10,3.334399e+11,PHL,LMC,111,81.64,5106638,5417915,4874611,5049750,20448914,19.889999,4067289
2,2019,3.768233e+11,3413.847924,13.0,3.355792,110380804.0,Philippines,1.264540e+10,3.641779e+11,PHL,LMC,111,81.64,5140782,5463479,4922457,5119620,20646338,17.469999,3606916
3,2020,3.617511e+11,3224.422551,13.0,3.734811,112190977.0,Philippines,1.351072e+10,3.482404e+11,PHL,LMC,111,81.64,5169706,5501379,4976624,5204858,20852567,21.049999,4389466
4,2021,3.940864e+11,3460.530963,13.0,3.731452,113880328.0,Philippines,1.470515e+10,3.793813e+11,PHL,LMC,111,81.64,5209594,5551557,5017104,5276578,21054833,17.469999,3678280


In [61]:
Singapore_info = negara_info[negara_info['Nama Negara'] == 'Singapore']
Singapore_info.drop('Nama Negara', axis=1, inplace=True)
Singapore_merge = pd.merge(copy_data_Singapore, Singapore_info, on="Year")
Singapore_merge

<ipython-input-61-de3a05f6707f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Singapore_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,3.431934e+11,61150.727197,6.0,2.768260,5612253.0,Singapore,9.500484e+09,3.336929e+11,SGP,HIC,3,105.89,151041,177869,158604,228010,715524,4.50,32199
1,2018,3.769981e+11,66859.338345,6.0,2.855980,5638676.0,Singapore,1.076699e+10,3.662312e+11,SGP,HIC,3,105.89,141606,161751,183833,227108,714298,4.30,30715
2,2019,3.754727e+11,65831.189431,6.0,2.736210,5703569.0,Singapore,1.027372e+10,3.651990e+11,SGP,HIC,3,105.89,133526,147458,181428,224384,686796,6.20,42582
3,2020,3.452959e+11,60729.450349,6.0,2.506660,5685807.0,Singapore,8.655395e+09,3.366405e+11,SGP,HIC,3,105.89,125446,134471,174150,214480,648547,4.46,28926
4,2021,3.969869e+11,72794.003023,6.0,2.806462,5453566.0,Singapore,1.114129e+10,3.858456e+11,SGP,HIC,3,105.89,115483,121409,158604,193205,588701,6.20,36500


In [62]:
Thailand_info = negara_info[negara_info['Nama Negara'] == 'Thailand']
Thailand_info.drop('Nama Negara', axis=1, inplace=True)
Thailand_merge = pd.merge(copy_data_Thailand, Thailand_info, on="Year")
Thailand_merge

<ipython-input-62-a0dfdebd62dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Thailand_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,4.563570e+11,6436.791746,9.0,3.465322,70898202.0,Thailand,1.581424e+10,4.405427e+11,THA,UMC,64,88.87,2210687,2348610,2299678,2589017,9447992,15.59,1472942
1,2018,5.067546e+11,7124.564544,9.0,3.198600,71127802.0,Thailand,1.620905e+10,4.905456e+11,THA,UMC,64,88.87,2137885,2271091,2486408,2589003,9484387,14.78,1401793
2,2019,5.440811e+11,7630.039610,9.0,3.020620,71307763.0,Thailand,1.643462e+10,5.276464e+11,THA,UMC,64,88.87,2084713,2210945,2448824,2563565,9308047,14.75,1372937
3,2020,4.996818e+11,6990.935503,9.0,3.145711,71475664.0,Thailand,1.571854e+10,4.839632e+11,THA,UMC,64,88.87,2046609,2165405,2384263,2508881,9105158,15.13,1377611
4,2021,5.059470e+11,7066.190546,9.0,3.207563,71601103.0,Thailand,1.622857e+10,4.897185e+11,THA,UMC,64,88.87,2016183,2129838,2299678,2429368,8875067,14.75,1309073


In [63]:
Vietnam_info = negara_info[negara_info['Nama Negara'] == 'Vietnam']
Vietnam_info.drop('Nama Negara', axis=1, inplace=True)
Vietnam_merge = pd.merge(copy_data_Vietnam, Vietnam_info, on="Year")
Vietnam_merge

<ipython-input-63-907557f35283>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vietnam_info.drop('Nama Negara', axis=1, inplace=True)


,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,2.813536e+11,2992.071742,10.0,4.085540,94033048.0,Vietnam,1.149481e+10,2.698588e+11,VNM,LMC,60,89.53,3346138,3512191,3355424,3923298,14137051,11.12,1572041
1,2018,3.101065e+11,3267.225009,10.0,4.167450,94914330.0,Vietnam,1.292353e+10,2.971829e+11,VNM,LMC,60,89.53,3351137,3528079,3675544,3776712,14331472,10.09,1446046
2,2019,3.343653e+11,3491.091274,10.0,4.061970,95776716.0,Vietnam,1.358182e+10,3.207834e+11,VNM,LMC,60,89.53,3370097,3560243,3528095,3638419,14096854,14.93,2104661
3,2020,3.466158e+11,3586.347302,10.0,4.111617,96648685.0,Vietnam,1.425151e+10,3.323642e+11,VNM,LMC,60,89.53,3386916,3592890,3418117,3537280,13935203,14.18,1976012
4,2021,3.661376e+11,3756.489121,10.0,4.106644,97468029.0,Vietnam,1.503597e+10,3.511016e+11,VNM,LMC,60,89.53,3386514,3610960,3355424,3483844,13836742,14.93,2065826


In [64]:
list_merge = [Brunei_merge, Myanmar_merge, Cambodia_merge, TimorLeste_merge, 
          Indonesia_merge, Laos_merge, Malaysia_merge, Philippines_merge, 
          Singapore_merge, Thailand_merge, Vietnam_merge]
df_merge = pd.concat(list_merge)
df_merge.head()

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ,"Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male",Total Youth,"Share of youth not in education, employment or training, total (% of youth population)","Total of youth not in education, employment or training"
0,2017,1.212810e+10,28186.803027,9.0,3.8893,430276.0,Brunei,4.716984e+08,1.165641e+10,BRN,HIC,74,87.58,16815,18112,16545,17857,69329,19.969999,13846
1,2018,1.356735e+10,31241.453955,9.0,3.8893,434274.0,Brunei,5.276750e+08,1.303968e+10,BRN,HIC,74,87.58,16852,18002,16669,17772,69295,20.719999,14358
2,2019,1.346942e+10,30748.737487,9.0,3.8893,438048.0,Brunei,5.238662e+08,1.294556e+10,BRN,HIC,74,87.58,16777,17815,16575,17742,68909,18.680000,12873
3,2020,1.200583e+10,27179.411985,9.0,3.8893,441725.0,Brunei,4.669426e+08,1.153888e+10,BRN,HIC,74,87.58,16574,17546,16557,17786,68463,21.430000,14672
4,2021,1.400657e+10,31449.076562,9.0,3.8893,445373.0,Brunei,5.447575e+08,1.346181e+10,BRN,HIC,74,87.58,16361,17260,16545,17850,68016,18.680000,12706


In [65]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 4
Data columns (total 20 columns):
 #   Column                                                                                  Non-Null Count  Dtype  
---  ------                                                                                  --------------  -----  
 0   Year                                                                                    55 non-null     object 
 1   PDB dalam USD                                                                           55 non-null     float64
 2   Pertumbuhan pendapatan negara                                                           55 non-null     float64
 3   Durasi Wajib Belajar                                                                    55 non-null     float64
 4   Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)                             55 non-null     float64
 5   Total Populasi                                                            

# Ambil Dataset

In [271]:
df_merge.to_csv('df_merge.csv')

# Analisis

In [ ]:
df_merge

In [ ]:
df_merge[['Nama Negara', 'Total of youth not in education, employment or training','Year', 'Total pengeluaran pemerintah untuk pendidikan']]

In [277]:
df_merge[df_merge['Year']=='2017']['Total pengeluaran pemerintah untuk pendidikan']

0    4.716984e+08
0    1.429522e+09
0    5.136576e+08
0    8.209218e+07
0    2.711682e+10
0    4.264330e+08
0    1.491948e+10
0    1.304448e+10
0    9.500484e+09
0    1.581424e+10
0    1.149481e+10
Name: Total pengeluaran pemerintah untuk pendidikan, dtype: float64

In [ ]:
df_2 = df_merge[['Nama Negara','Pertumbuhan pendapatan negara','Year']]
df_2.head(10)

In [103]:
avg_list = df_2.groupby(['Nama Negara', 'Year']).mean().groupby('Nama Negara')['Pertumbuhan pendapatan negara'].mean().tolist()

In [ ]:
df_2.drop(['Pertumbuhan pendapatan negara','Year'], axis=1, inplace=True)
df_2.drop_duplicates(inplace=True)
df_2['avg'] = avg_list
df_2

In [78]:
df_2.info()

<class 'pandas.core.series.Series'>
Index: 11 entries, Brunei to Vietnam
Series name: Pertumbuhan pendapatan negara
Non-Null Count  Dtype  
--------------  -----  
11 non-null     float64
dtypes: float64(1)
memory usage: 476.0+ bytes


In [57]:
df_merge.head()

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,Total pengeluaran pemerintah SELAIN pendidikan,code,Income Level,Rank IQ,IQ
0,2017,1.212810e+10,28186.803027,9.0,3.8893,430276.0,Brunei,4.716984e+08,1.165641e+10,BRN,HIC,74,87.58
1,2018,1.356735e+10,31241.453955,9.0,3.8893,434274.0,Brunei,5.276750e+08,1.303968e+10,BRN,HIC,74,87.58
2,2019,1.346942e+10,30748.737487,9.0,3.8893,438048.0,Brunei,5.238662e+08,1.294556e+10,BRN,HIC,74,87.58
3,2020,1.200583e+10,27179.411985,9.0,3.8893,441725.0,Brunei,4.669426e+08,1.153888e+10,BRN,HIC,74,87.58
4,2021,1.400657e+10,31449.076562,9.0,3.8893,445373.0,Brunei,5.447575e+08,1.346181e+10,BRN,HIC,74,87.58


In [ ]:
list_coba = [df_merge]
list_coba

In [ ]:
negara_info['Nama Negara'].tolist()

In [ ]:
# Berdasarkan list nama negara
list_countries = negara_info['Nama Negara'].tolist()

# buat dataframe untuk tahun 2017 dan 2021
df_2017 = pd.DataFrame()
df_2021 = pd.DataFrame()

for i, df in enumerate(list_merge):
    # pilih data Total Populasi dan Tahunnya
    df_2017[list_countries[i]] = df[df['Year'] == '2017']["Total Populasi"]
    df_2021[list_countries[i]] = df[df['Year'] == '2021']["Total Populasi"]

# Hasil Dataframe nama negara menjadi kolom, untuk mempermudah visualisasi perlu dilakukan transpose
df_2017 = df_2017.T
df_2021 = df_2021.T

# Mengganti nama kolom
df_2017 = df_2017.rename(columns={0 : 2017})
df_2021 = df_2021.rename(columns={4 : 2021})

# Merge dataset 2017 dan 2021
df_1721= df_2017.join(df_2021)

# Nama negara yang menjadi index di masukkan ke kolom baru (untuk mempermudah visualisasi)
df_1721['Negara'] = df_1721.index

# Drop index
df_1721.reset_index(drop=True)

print("Total Populasi setiap negara tahun 2017 dan 2021: ")
display(df_1721)

In [ ]:
df[df['Year'] == '2017']["Total Populasi"]

0    94033048.0
Name: Total Populasi, dtype: float64

In [ ]:
list_countries = negara_info['Nama Negara'].tolist()

# buat dataframe untuk tahun 2017 dan 2021
df_2017 = pd.DataFrame()
df_2021 = pd.DataFrame()

for i, df in enumerate(list_merge):
    # pilih data Total Populasi dan Tahunnya
    df_2017[list_countries[i]] = df[df['Year'] == '2017']["Total Populasi"]
    df_2021[list_countries[i]] = df[df['Year'] == '2017']['Pertumbuhan pendapatan negara']

coba = pd.concat([df_2017,df_2021]).reset_index()
coba.drop('index',axis=1, inplace=True)
coba.reset_index()

,index,Brunei,Myanmar,Cambodia,Timor-Leste,Indonesia,Laos,Malaysia,Philippines,Singapore,Thailand,Vietnam
0,0,430276.000000,5.228834e+07,1.583069e+07,1.243235e+06,2.644989e+08,6.997917e+06,3.197581e+07,1.067385e+08,5.612253e+06,7.089820e+07,9.403305e+07
1,1,28186.803027,1.175203e+03,1.400899e+03,1.283526e+03,3.839785e+03,2.439463e+03,9.979801e+03,3.077436e+03,6.115073e+04,6.436792e+03,2.992072e+03


In [ ]:
coba.columns.values.tolist()

['Brunei',
 'Myanmar',
 'Cambodia',
 'Timor-Leste',
 'Indonesia',
 'Laos',
 'Malaysia',
 'Philippines',
 'Singapore',
 'Thailand',
 'Vietnam']

In [ ]:
# Berdasarkan Indicator
df_merge[df_merge['Year'] == '2021']

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,code,Income Level,Rank IQ,IQ
4,2021,1.400657e+10,31449.076562,9.0,3.889300,445373.0,Brunei,5.447575e+08,BRN,HIC,74,87.58
9,2021,6.509175e+10,1209.926942,5.0,2.207897,53798084.0,Myanmar,1.437159e+09,MMR,LMC,52,91.18
14,2021,2.696106e+10,1625.235020,12.0,2.729287,16589023.0,Cambodia,7.358447e+08,KHM,LMC,15,99.75
19,2021,3.621222e+09,2741.393931,9.0,4.563622,1320942.0,Timor-Leste,1.652589e+08,TLS,LMC,132,78.49
24,2021,1.186093e+12,4332.709281,9.0,3.000458,273753191.0,Indonesia,3.558822e+10,IDN,LMC,130,78.49
29,2021,1.882715e+10,2535.623432,9.0,2.402019,7425057.0,Laos,4.522316e+08,LAO,LMC,114,80.99
34,2021,3.729810e+11,11109.261839,6.0,4.306885,33573874.0,Malaysia,1.606386e+10,MYS,UMC,73,87.58
39,2021,3.940864e+11,3460.530963,13.0,3.731452,113880328.0,Philippines,1.470515e+10,PHL,LMC,111,81.64
44,2021,3.969869e+11,72794.003023,6.0,2.806462,5453566.0,Singapore,1.114129e+10,SGP,HIC,3,105.89
49,2021,5.059470e+11,7066.190546,9.0,3.207563,71601103.0,Thailand,1.622857e+10,THA,UMC,64,88.87


In [ ]:
df_merge[df_merge['Year']=='2021'].sort_values('Rank IQ')

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,code,Income Level,Rank IQ,IQ
44,2021,3.969869e+11,72794.003023,6.0,2.806462,5453566.0,Singapore,1.114129e+10,SGP,HIC,3,105.89
14,2021,2.696106e+10,1625.235020,12.0,2.729287,16589023.0,Cambodia,7.358447e+08,KHM,LMC,15,99.75
9,2021,6.509175e+10,1209.926942,5.0,2.207897,53798084.0,Myanmar,1.437159e+09,MMR,LMC,52,91.18
54,2021,3.661376e+11,3756.489121,10.0,4.106644,97468029.0,Vietnam,1.503597e+10,VNM,LMC,60,89.53
49,2021,5.059470e+11,7066.190546,9.0,3.207563,71601103.0,Thailand,1.622857e+10,THA,UMC,64,88.87
34,2021,3.729810e+11,11109.261839,6.0,4.306885,33573874.0,Malaysia,1.606386e+10,MYS,UMC,73,87.58
4,2021,1.400657e+10,31449.076562,9.0,3.889300,445373.0,Brunei,5.447575e+08,BRN,HIC,74,87.58
39,2021,3.940864e+11,3460.530963,13.0,3.731452,113880328.0,Philippines,1.470515e+10,PHL,LMC,111,81.64
29,2021,1.882715e+10,2535.623432,9.0,2.402019,7425057.0,Laos,4.522316e+08,LAO,LMC,114,80.99
24,2021,1.186093e+12,4332.709281,9.0,3.000458,273753191.0,Indonesia,3.558822e+10,IDN,LMC,130,78.49


In [ ]:
country = ['Indonesia', 'Siangpore']

In [ ]:
df_merge[df_merge['Nama Negara'].isin(country)]

,Year,PDB dalam USD,Pertumbuhan pendapatan negara,Durasi Wajib Belajar,"Pengeluaran pemerintah untuk pendidikan, total (% dari PDB)",Total Populasi,Nama Negara,Total pengeluaran pemerintah untuk pendidikan,code,Income Level,Rank IQ,IQ
20,2017,1.015619e+12,3839.785069,9.0,2.669980,264498852.0,Indonesia,2.711682e+10,IDN,LMC,130,78.49
21,2018,1.042272e+12,3902.661668,9.0,3.000000,267066843.0,Indonesia,3.126815e+10,IDN,LMC,130,78.49
22,2019,1.119100e+12,4151.227543,9.0,2.841850,269582878.0,Indonesia,3.180314e+10,IDN,LMC,130,78.49
23,2020,1.058689e+12,3894.272202,9.0,3.490000,271857970.0,Indonesia,3.694824e+10,IDN,LMC,130,78.49
24,2021,1.186093e+12,4332.709281,9.0,3.000458,273753191.0,Indonesia,3.558822e+10,IDN,LMC,130,78.49
